In [2]:
import numpy as np
import random
def generate_state(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd):
    """
    生成一个系统的随机状态，该系统负责将任务分配给员工。
    
    参数:
    I_citys (int): 城市的数量。
    L_levels (int): 等级的数量。
    W_workdays (int): 员工距离放假的最大工作天数, 一般为7。
    M_servers (int): 员工的数量。
    x_max_task_num (int): 每个城市的最大任务数(本函数中未直接使用)。
    H_home_of_server (list[int]): 代表每个员工家所在城市的列表。
    lambd (np.array): 一个二维数组 (I_citys x_max_task_num L_levels)，代表每个城市和等级的任务到达率。
    
    返回:
    tuple: 包含任务分布矩阵和员工状态列表的元组。

    城市中的任务矩阵，员工状态列表[(员工所在地，距离放假时间)]
    """

    # 生成任务分布矩阵 (n_il)，其维度为 I_citys x_max_task_num L_levels  
    n_il = np.zeros((I_citys, L_levels), dtype=int)  # 用零初始化矩阵
    for i_city in range(I_citys):  # 遍历城市
        for l_level in range(L_levels):  # 遍历等级
            # 为每个城市和等级分配一个基于泊松分布的随机数, 表示随机状态生成的任务数量？# TODO
            n_il[i_city, l_level] = np.random.poisson(lambd[i_city, l_level])
    S0_tasks = n_il  # 任务分布矩阵 (I_citys x_max_task_num L_levels), [0, +∞)
    
    # 初始化一个列表来保存每个员工的状态，给
    S1_servers = []
    for m_server in range(M_servers):  # 遍历所有员工

        # 为员工 'm_server' 随机选择距离放假的工作日数，取值范围 [0, W_workdays]
        w_m = np.random.randint(0, W_workdays + 1)

        # 根据距离放假天数w_m，得到位置i_m，如果距离放假0，则在家里，否则随机一个位置。
        # i_m [1, I_citys]
        if w_m == W_workdays:  # 如果距离放假时间为 0 天，即今天放假，则员工所在城市为家所在城市
            i_m = H_home_of_server[m_server]
        else:  # 否则，为员工随机选择一 个非家乡城市工作
            i_m = np.random.randint(1, I_citys + 1)
            
        # 将员工的状态作为元组（城市，距离放假的工作日数）添加到列表中
        S1_servers.append((i_m, w_m))
    
    # 将任务分布矩阵和员工状态列表合并成一个状态元组
    S = (S0_tasks, S1_servers)    # ((I_citys x_max_task_num L_levels), (M_servers x_max_task_num 1))

    return S  # 返回生成的状态

In [3]:
I_citys = 26
L_levels = 5
W_workdays = 6
M_servers = 40
x_max_task_num = 2
H_home_of_server = [1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,]
lambd = np.random.rand(I_citys, L_levels)
func1 = generate_state
state = func1(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd)
print(state)

(array([[1, 0, 0, 1, 3],
       [0, 0, 0, 0, 1],
       [0, 1, 2, 0, 0],
       [0, 1, 1, 0, 1],
       [0, 1, 0, 0, 3],
       [1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 2, 0, 0, 0],
       [0, 0, 2, 2, 0],
       [2, 0, 0, 0, 0],
       [2, 2, 0, 0, 2],
       [3, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 2, 0],
       [3, 0, 2, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 2, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 2],
       [2, 0, 2, 0, 0],
       [2, 1, 0, 0, 0]]), [(16, 0), (14, 4), (19, 3), (5, 1), (5, 0), (6, 6), (7, 6), (5, 3), (8, 4), (16, 1), (8, 1), (7, 2), (4, 2), (11, 3), (16, 3), (22, 1), (15, 1), (8, 4), (14, 4), (21, 1), (6, 4), (11, 5), (3, 6), (15, 5), (22, 5), (12, 5), (14, 2), (17, 5), (11, 1), (14, 5), (21, 4), (20, 4), (14, 5), (18, 2), (22, 5), (21, 0), (19, 0), (22, 5), (12, 1), (10, 6)])


In [4]:
def aggreg_state(S, Z_cluster_num, X, M_servers, I_citys, L_levels):
    # 函数2
    # 定义一个函数，用于根据给定的参数将复杂的状态 S 压缩成一个简化的状态 barS。
    # 输入:
    #   S: 当前状态，一个复杂的结构，包含两部分信息：
    #      - 一个数组，表示每个城市每个等级的数量(n_il)。
    #      - 一个列表，表示服务员和他们服务的城市及工作日(i_m, w_m)
    #   Z_cluster_num: 一个整数，表示将城市分成多少个聚类。
    #   X: 一个整数，用于计算 N 矩阵中的元素值。
    #   M_servers: 服务员的总数。
    #   I_citys: 城市的总数。
    #   L_levels: 等级的总数。
    # 输出:
    #   barS: 一个元组，表示压缩后的状态，包含以下三个部分：
    #         - N: 一个二维数组，表示每个聚类的等级之和。
    #         - g: 一个数组，表示每个聚类的状态。
    #         - w: 一个整数，表示第一个服务员的工作日数。

    # 计算正在工作的服务员的数量, S[1][m_server][1]即S1_servers[m_server][1]即w_m
    barM = np.sum([1 for m_server in range(M_servers) if S[1][m_server][1] != 0]) # 距离放假时间不等于0
    # 根据城市数量和设定的簇数，将城市分成Z个簇
    cluster = split_list(I_citys, Z_cluster_num)
    # 计算实际的簇数，考虑到可能会有余数
    # num_cluster = divide_reminder(I_citys, Z_cluster_num) # 这个可以替换为下面的
    num_cluster = np.ceil(I_citys / Z_cluster_num).astype(int) # 向上取整
    # 初始化表示各簇状态的数组g
    g = np.zeros(num_cluster)
    
    # 压缩状态的第二部分：计算每个簇的状态
    for z_cluster in range(num_cluster):
        # 统计每个簇中有多少业务员正在工作 
        e_z = np.sum([1 for m_server in range(M_servers) if S[1][m_server][0] in cluster[z_cluster]])
        # 根据工作的业务员数量设置簇的状态
        if e_z == 0:
            g[z_cluster] = 0  # 无业务员工作
        elif e_z <= barM / num_cluster:
            g[z_cluster] = 1  # 工作业务员数量低于或等于平均值
        else:
            g[z_cluster] = 2  # 工作业务员数量高于平均值
    
    # 获取第一个业务员的工作量
    w = S[1][0][1]
    
    # 压缩状态的第一部分：计算每个簇中各等级的数量总和
    N = np.zeros((num_cluster, L_levels))  # 初始化N矩阵
    for z_cluster in range(num_cluster):
        for l in range(L_levels):
            # 对每个簇的每个等级，计算其数量总和，但不超过X
            N[z_cluster][l] = min(X, np.sum([S[0][i-1][l] for i in cluster[z_cluster]]))
            
    # 将计算出的N矩阵、簇的状态数组g和第一个业务员的工作量w组合成新的压缩状态barS
    barS = (N, g, w)        
    return barS

# 将城市列表平均分成Z个聚类
def split_list(I_citys, Z_cluster_num)->list:
    # 创建一个从1到I_citys的城市索引列表
    arr_city_idx = list(range(1, I_citys + 1))
    # 调用函数处理实际的分割
    return split_array_given_array(arr_city_idx, Z_cluster_num)



def split_array_given_array(arr_city_idx, Z_cluster_num)->list:
    """
    将输入数组分割成长度为 Z_cluster_num 的子数组列表。如果数组不能被 Z_cluster_num 整除，
    那么最后一个子数组将包含所有剩余的元素。
    
    参数:
    arr_city_idx (list): 需要被分割的输入数组。
    Z_cluster_num (int): 每个子数组的期望长度。
    
    返回值:
    list: 长度为 Z_cluster_num 的子数组列表，除了可能的最后一个子数组，它包含所有剩余的元素。
    """
    result = []  # 结果列表，用来存储所有的子数组
    quotient = len(arr_city_idx) // Z_cluster_num  # 计算整除的商，即完整子数组的数量
    remainder = len(arr_city_idx) % Z_cluster_num  # 计算余数，即最后一个子数组的元素数量
    
    # 划分可以整除的数组部分
    for i in range(quotient):
        sub_array = arr_city_idx[i * Z_cluster_num:(i + 1) * Z_cluster_num]  # 获取从 i*Z_cluster_num 到 (i+1)*Z_cluster_num 的子数组
        result.append(sub_array)  # 将子数组添加到结果列表中
    
    # 如果有余数，则处理剩余部分
    if remainder > 0:
        sub_array = arr_city_idx[-remainder:]  # 获取数组最后余数个元素形成的子数组
        result.append(sub_array)  # 将子数组添加到结果列表中
    
    return result  # 返回结果列表

def divide_reminder(num, divisor):
    """
    将一个整数除以另一个整数，并将结果向上取整。
    
    参数:
    num (int): 被除数。
    divisor (int): 除数。
    
    返回值:
    int: 向上取整后的商。
    """
    quotient = num // divisor  # 计算整除的商
    remainder = num % divisor  # 计算余数
    
    # 如果存在余数，则将商向上取整
    if remainder > 0:
        quotient += 1  # 余数大于0，商加一
    
    return quotient  # 返回向上取整后的商

In [5]:
Z_cluster_num=3
X=3
M_servers=40
I_citys=26
L_levels=5
barS=aggreg_state(state, Z_cluster_num, X, M_servers, I_citys, L_levels)
print(barS)

(array([[1., 1., 2., 1., 3.],
       [1., 2., 1., 0., 3.],
       [1., 2., 0., 0., 0.],
       [3., 2., 2., 2., 2.],
       [3., 0., 2., 2., 1.],
       [3., 0., 2., 1., 1.],
       [0., 0., 2., 3., 1.],
       [0., 0., 3., 0., 2.],
       [3., 1., 2., 0., 0.]]), array([1., 2., 2., 2., 2., 2., 2., 1., 0.]), 0)


In [16]:
S = state
# (位置i_m, 放假天数w_m)
rest_day_L = [server_restday[1] for server_restday in state[1]]
rest_day_L
# print(f"{S=}\n {mathscr_L=}")

[0,
 4,
 3,
 1,
 0,
 6,
 6,
 3,
 4,
 1,
 1,
 2,
 2,
 3,
 3,
 1,
 1,
 4,
 4,
 1,
 4,
 5,
 6,
 5,
 5,
 5,
 2,
 5,
 1,
 5,
 4,
 4,
 5,
 2,
 5,
 0,
 0,
 5,
 1,
 6]

In [14]:
def f_4(S, mathscr_L):
    # 假设 len(mathscr_L) = 7
    # 假设 S 和 mathscr_L 的数据结构如所描述
    # S 是一个二元组 (S0_tasks, S1_servers)，其中 S0_tasks 是任务矩阵
    # mathscr_L 是业务员的等级集合，例如 [l_1, l_2, ..., l_M]
    #S0 (I_citys x_max_task_num L_levels)
    # 首先计算 N_1 和 N_2
    L_levels = 5
    N_1 = [sum(1 for l_m, (i_m, w_m) in zip(mathscr_L, S[1]) if l_m == j and w_m != 0) for j in range(1, L_levels+1)]
    N_2 = [sum(S[0][i][j] for i in range(len(S[0]))) for j in range(L_levels)]
    # print(f"{N_1=} {N_2=}")
    # 初始化分类后的等级列表
    mathcal_L = []
    current_class = []
    total_N_1 = 0
    total_N_2 = 0

    for j in range(1, L_levels+1):
        total_N_1 += N_1[j-1]
        total_N_2 += N_2[j-1]
        current_class.append(j)

        if total_N_1 <= total_N_2:
            # 当 N_1 总和小于等于 N_2 总和时，终止当前类的添加
            mathcal_L.append(tuple(current_class))
            current_class = []
            total_N_1 = 0
            total_N_2 = 0

    if current_class:
        # 添加最后一个类
        mathcal_L.append(tuple(current_class))

    return mathcal_L, N_1, N_2


In [7]:
S = state
# (位置i_m, 放假天数w_m)
mathscr_L = [1,1,1,2,3,4,5,2,3,5, 1,1,1,2,3,4,5,2,3,5,1,1,1,2,3,4,5,2,3,5,1,1,1,2,3,4,5,2,3,5,]
# print(f"{S=}\n {mathscr_L=}")
mathcal_L, N_1, N_2 = f_4(S, mathscr_L)
print(mathcal_L, N_1, N_2)

[(1, 2), (3,), (4,), (5,)] [11, 8, 8, 4, 8] [10, 20, 16, 12, 9]


In [8]:
len(S[0]),len(S[1])

(26, 40)

In [9]:
import pulp

def func3_fix(S, L_mathscr, H_home_of_server, r1, c1, c2):
    """
    生成状态 S 到决策 A 的函数，通过解决线性规划问题来最大化收益 R(S, A)。

    参数:
    S (tuple): 当前状态，包含任务矩阵和服务员信息。
    L_mathscr (list): 服务员的等级列表。
    H_home_of_server (list): 服务员的家位置列表。
    r1 (list): 每个等级的收益列表。
    c1 (list of list): I×I 的成本矩阵。
    c2 (float): 常数成本。

    返回:
    list: 最优决策 A，包含每个服务员的位置和等级。
    """
    n_il, servers_info = S
    
    # n_il, servers_info = S
    # M_servers = len(servers_info)  # 服务员数量
    # I_citys = len(n_il)          # 城市数量
    # # L_max = len(r1) - 1    # 最大等级

    M_servers = len(servers_info)  # 服务员数量
    I_citys = len(n_il)          # 城市数量
    L_max = max(L_mathscr) # 最大等级
    # print(f"{M_servers=} {I_citys=} {L_max=} {L_mathscr=}")

    # 创建问题实例
    prob = pulp.LpProblem("Optimal_Server_Assignment", pulp.LpMaximize)

    # 定义决策变量 y_{mil} 为二元变量
    y = pulp.LpVariable.dicts("y", 
                              ((m, i, l) for m in range(M_servers) for i in range(I_citys) for l in range(L_max+1)), 
                              cat=pulp.LpBinary)

    # M_servers=40 
    # I_citys=26 
    # L_max=5
    # 目标函数
    # 假设 L_max 和 r1 已经定义
    print("L_max:", L_max)
    print("Length of r1:", len(r1))

    # 遍历 r1 的元素，从索引 0 到 L_max（包含 L_max）
    # try:
    #     for l in range(L_max + 1):  # 注意这里使用 L_max + 1
    #         print(f"r1[{l}] = {r1[l]}")  # 打印每个元素来验证访问是否成功
    # except IndexError as e:
    #     print(f"IndexError: Trying to access r1[{l}], but it's out of range. Length of r1 is {len(r1)}.")
    prob += pulp.lpSum(
        r1[l1] * y[m1, i1, l1] - c1[servers_info[m1][0]-1][i1] * y[m1, i1, l1]
                       for m1 in range(M_servers) for i1 in range(I_citys)
                       for l1 in range(0, L_max+1))\
                        - c2 * pulp.lpSum(n_il[i][l-1] - pulp.lpSum(y[m, i, l] for m in range(M_servers)) for i in range(I_citys)
                       for l in range(1, L_max+1))

    # 添加约束
    # 每个服务员只能分配到一个地点和等级
    for m in range(M_servers):
        prob += pulp.lpSum(y[m, i, l] for i in range(I_citys) for l in range(L_max+1)) == 1

    # 服务员不工作时，分配到家乡的等级0
    for m, (im, wm) in enumerate(servers_info):
        if wm == 0:
            prob += y[m, H_home_of_server[m], 0] == 1

    # 服务员工作时，必须分配到合适的等级和城市
    for m, (im, wm) in enumerate(servers_info):
        if wm > 0:
            prob += pulp.lpSum(y[m, i, l] for i in range(I_citys) for l in range(L_mathscr[m], L_max+1) if n_il[i][l-1] > 0) == 1

    # 资源使用不超过可用数量
    for i in range(I_citys):
        for l in range(1, L_max+1):
            prob += pulp.lpSum(y[m, i, l] for m in range(M_servers)) <= n_il[i][l-1]

    # 求解问题
    prob.solve()

    # 解析结果
    result = [(m, i, l) for m in range(M_servers) for i in range(I_citys) for l in range(L_max+1) if pulp.value(y[m, i, l]) == 1]
    return result


In [10]:
S = state
n_il, servers_info = S
L_mathscr = mathscr_L
H_home_of_server = H_home_of_server
r1  = [0, 3500, 3000, 2500, 2000, 1500]
# 假设 I_citys 是城市的数量，这个值应该根据你的具体情况来设置
I_citys = len(n_il)  # 以 n_il 变量中元素的数量来确定城市数量

# 生成随机的成本矩阵 c1
c1 = [[0 if i == j else random.randint(100, 500) for j in range(I_citys)] for i in range(I_citys)]
c2 = 100
A =func3_fix(S, L_mathscr, H_home_of_server, r1, c1, c2)

L_max: 5
Length of r1: 6


In [11]:
len(A), A

(40,
 [(0, 16, 1),
  (1, 7, 1),
  (2, 8, 1),
  (3, 12, 2),
  (4, 9, 3),
  (5, 15, 4),
  (6, 23, 5),
  (7, 21, 2),
  (8, 9, 3),
  (9, 16, 5),
  (10, 21, 1),
  (11, 17, 1),
  (12, 23, 1),
  (13, 25, 2),
  (14, 0, 3),
  (15, 9, 4),
  (16, 23, 5),
  (17, 7, 2),
  (18, 0, 3),
  (19, 19, 5),
  (20, 25, 1),
  (21, 21, 1),
  (22, 25, 2),
  (23, 9, 2),
  (24, 15, 3),
  (25, 6, 4),
  (26, 24, 5),
  (27, 7, 2),
  (28, 15, 3),
  (29, 22, 5),
  (30, 23, 1),
  (31, 20, 1),
  (32, 3, 0),
  (33, 15, 2),
  (34, 5, 3),
  (35, 18, 4),
  (36, 24, 5),
  (37, 1, 2),
  (38, 8, 3),
  (39, 25, 5)])

In [12]:
mathcal_L

[(1, 2), (3,), (4,), (5,)]

In [13]:
mathcal_L

[(1, 2), (3,), (4,), (5,)]

In [14]:
import pulp

def func5(S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1, c1, c2):
    """
    生成状态 S 到决策 Y 的函数,通过解决线性规划问题来最大化收益 R(S, Y)。

    参数:
    S (tuple): 当前状态,包含任务矩阵和服务员信息。
    mathcal_L (list): 分类后的等级列表。
    mathscr_L (list): 所有服务员的等级列表
    N_1 (list): 每个等级的服务员数量。
    N_2 (list): 每个等级的任务数量。
    H_home_of_server (list): 服务员的家位置列表。
    r1 (list): 每个等级的收益列表。
    c1 (list of list): I×I 的成本矩阵。
    c2 (float): 常数成本。

    返回:
    list: 最优决策 Y,包含每个服务员的位置和等级。
    total_revenue, 总收益
    """
    n_il, servers_info = S
    M_servers = len(servers_info)  # 服务员数量
    I_citys = len(n_il)          # 城市数量
    L_max = [max(l) for l in mathcal_L]  # 最大等级
    print("M_servers, I_citys, L_max, N_1, N_2, mathcal_L, mathscr_L", M_servers, I_citys, L_max, N_1, N_2, mathcal_L, mathscr_L)
    # 步骤1:安排放假的员工回家
    C_h = sum(c1[servers_info[m][0]-1][H_home_of_server[m]] for m in range(M_servers) if servers_info[m][1] == 0)
    
    total_revenue = -C_h  # 初始化总收益为负的回家成本

    Y = [None] * M_servers  # 初始化最优决策 Y
    
    Y_set = []
    # 步骤2:对每个等级类独立进行员工分配
    for L_set, l_max_L in zip(mathcal_L, L_max):
        print("L_set ", L_set)
        M_servers_L = [m for m in range(M_servers) if servers_info[m][1] > 0 and mathscr_L[m] in L_set]  # 该等级类下工作的员工集合
        I_citys_L = [i for i in range(I_citys) if any(n_il[i][l-1] > 0 for l in L_set)]  # 该等级类下有任务需求的城市集合
        print("M_servers_L, I_citys_L", M_servers_L, I_citys_L)
        # 创建问题实例
        prob = pulp.LpProblem(f"Optimal_Server_Assignment_Level_{L_set}", pulp.LpMaximize)

        # 定义决策变量 y_{mil} 为二元变量
        y = pulp.LpVariable.dicts("y", 
                                  ((m, i, l) for m in M_servers_L for i in I_citys_L for l in L_set), 
                                  cat=pulp.LpBinary)

        # 目标函数
        if sum(N_1[l-1] for l in L_set) <= sum(N_2[l-1] for l in L_set):  # 等级类型为"≤"
            prob += pulp.lpSum(
                r1[l1-1] * y[m1, i1, l1] - c1[servers_info[m1][0]-1][i1] * y[m1, i1, l1]
                for m1 in M_servers_L for i1 in I_citys_L for l1 in L_set) \
                - c2 * pulp.lpSum(n_il[i][l-1] - pulp.lpSum(y[m, i, l] for m in M_servers_L) 
                                  for i in I_citys_L for l in L_set)
        else:  # 等级类型为">"
            prob += pulp.lpSum(
                r1[l1-1] * y[m1, i1, l1] - c1[servers_info[m1][0]-1][i1] * y[m1, i1, l1]
                for m1 in M_servers_L for i1 in I_citys_L for l1 in L_set)

        # 添加约束
        for m in M_servers_L: 
            # 每个工作中的服务员 m,要求其被分配到城市 i 提供的服务等级 l 必须不低于他自身的服务等级 L_mathscr[m]
            # 且只能被分配到一个城市提供一种等级的服务。
            prob += pulp.lpSum(y[m, i, l] for i in I_citys_L for l in L_set if l >= mathscr_L[m]) == 1

        for i in I_citys_L:
            for l in L_set:
                if sum(N_1[l-1] for l in L_set) <= sum(N_2[l-1] for l in L_set):  # 等级类型为"≤"
                    prob += pulp.lpSum(y[m, i, l] for m in M_servers_L) <= n_il[i][l-1]
                else:  # 等级类型为">"
                    prob += pulp.lpSum(y[m, i, l] for m in M_servers_L) == n_il[i][l-1]

        # 求解问题
        prob.solve()

        
        for m in M_servers_L:
            for i in I_citys_L:
                for l in L_set:
                    Y_sub_set = []
                    if pulp.value(y[m, i, l]) == 1:
                        Y[m] = (i+1, l)  # 城市编号从1开始
                        break

        # 更新总收益
        total_revenue += pulp.value(prob.objective)

        # 提取结果
        # Y_L = [(i, l) for m in M_servers_L for i in I_citys_L for l in L_set if y[m, i, l].value() == 1]
        # Y_set.append(Y_L)

    
    # 步骤3:安排放假的员工
    for m in range(M_servers):
        if servers_info[m][1] == 0:
            Y[m] = (H_home_of_server[m]+1, 0)  # 城市编号从1开始
    # # 步骤3:计算总收益
    # R = sum(prob.objective.value() for L in L_set) - C_h
    #     Y_set = []
        # 解析结果


    return Y, total_revenue

In [15]:
Y, total_revenue = func5(S,mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1, c1, c2)

M_servers, I_citys, L_max, N_1, N_2, mathcal_L, mathscr_L 40 26 [2, 3, 4, 5] [11, 8, 8, 4, 8] [10, 20, 16, 12, 9] [(1, 2), (3,), (4,), (5,)] [1, 1, 1, 2, 3, 4, 5, 2, 3, 5, 1, 1, 1, 2, 3, 4, 5, 2, 3, 5, 1, 1, 1, 2, 3, 4, 5, 2, 3, 5, 1, 1, 1, 2, 3, 4, 5, 2, 3, 5]
L_set  (1, 2)
M_servers_L, I_citys_L [0, 1, 2, 3, 7, 10, 11, 12, 13, 17, 20, 21, 22, 23, 27, 30, 31, 33, 37] [1, 6, 7, 8, 9, 12, 13, 15, 16, 17, 20, 21, 23, 25]
L_set  (3,)
M_servers_L, I_citys_L [4, 8, 14, 18, 24, 28, 34, 38] [0, 2, 5, 6, 8, 9, 13, 15, 20]
L_set  (4,)
M_servers_L, I_citys_L [5, 15, 25, 35] [1, 6, 8, 9, 11, 13, 15, 16, 18]
L_set  (5,)
M_servers_L, I_citys_L [6, 9, 16, 19, 26, 29, 36, 39] [16, 19, 22, 23, 24, 25]


C:\Users\dylan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [16]:
Y, len(Y), total_revenue

([(24, 2),
  (24, 2),
  (9, 2),
  (24, 2),
  (10, 3),
  (16, 4),
  (24, 5),
  (22, 1),
  (10, 3),
  (17, 5),
  (13, 2),
  (16, 2),
  (14, 2),
  (26, 2),
  (1, 3),
  (10, 4),
  (24, 5),
  (8, 1),
  (1, 3),
  (20, 5),
  (26, 2),
  (7, 2),
  (26, 2),
  (7, 2),
  (16, 3),
  (7, 4),
  (25, 5),
  (8, 2),
  (16, 3),
  (23, 5),
  (10, 2),
  (24, 2),
  (4, 0),
  (24, 2),
  (6, 3),
  (19, 4),
  (25, 5),
  (2, 2),
  (9, 3),
  (26, 5)],
 40,
 108293.0)

In [17]:
def func6(S, mathcal_L, mathscr_L, N_1, N_2):
    """
    生成状态 S 的决策空间 A,满足约束条件。

    参数:
    S (tuple): 当前状态,包含任务矩阵和服务员信息。
    mathcal_L (list): 分类后的等级列表。
    mathscr_L (list): 所有服务员的等级列表
    N_1 (list): 每个等级的服务员数量。
    N_2 (list): 每个等级的任务数量。

    返回:
    list: 决策空间 A,包含每个服务员的所有可能决策。
    """
    n_il, servers_info = S
    M_servers = len(servers_info)  # 服务员数量
    I_citys = len(n_il)          # 城市数量

    A = [[] for _ in range(M_servers)]  # 初始化决策空间 A

    # 对每个服务员生成可能的决策
    for m in range(M_servers):
        if servers_info[m][1] == 0:  # 服务员 m 放假
            A[m].append((servers_info[m][0], 0))  # 放假的服务员只有一个决策,即回家
        else:  # 服务员 m 工作
            for L_set in mathcal_L:  # 遍历每个等级类
                if mathscr_L[m] in L_set:  # 如果服务员 m 的等级属于当前等级类
                    for i in range(I_citys):  # 遍历每个城市
                        for l in L_set:  # 遍历当前等级类的每个等级
                            if l >= mathscr_L[m]:  # 如果当前等级不低于服务员 m 的等级
                                if (sum(N_1[l-1] for l in L_set) <= sum(N_2[l-1] for l in L_set) and 
                                    sum(1 for a in A[m] if a[0] == i+1) < sum(n_il[i][l-1] for l in L_set)) or \
                                   (sum(N_1[l-1] for l in L_set) > sum(N_2[l-1] for l in L_set) and
                                    sum(1 for a in A[m] if a[0] == i+1 and a[1] == l) < n_il[i][l-1]):
                                    # 如果满足约束条件,则将决策添加到服务员 m 的决策空间中
                                    A[m].append((i+1, l))  # 城市编号从1开始

    return A

In [18]:
A_all = func6(S, mathcal_L, mathscr_L, N_1, N_2)
len(A_all)

40

In [19]:
A_all

[[(2, 1),
  (7, 1),
  (7, 2),
  (8, 1),
  (8, 2),
  (9, 1),
  (9, 2),
  (10, 1),
  (13, 1),
  (14, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (21, 1),
  (22, 1),
  (22, 2),
  (24, 1),
  (24, 2),
  (26, 1),
  (26, 2)],
 [(2, 1),
  (7, 1),
  (7, 2),
  (8, 1),
  (8, 2),
  (9, 1),
  (9, 2),
  (10, 1),
  (13, 1),
  (14, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (21, 1),
  (22, 1),
  (22, 2),
  (24, 1),
  (24, 2),
  (26, 1),
  (26, 2)],
 [(2, 1),
  (7, 1),
  (7, 2),
  (8, 1),
  (8, 2),
  (9, 1),
  (9, 2),
  (10, 1),
  (13, 1),
  (14, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (21, 1),
  (22, 1),
  (22, 2),
  (24, 1),
  (24, 2),
  (26, 1),
  (26, 2)],
 [(2, 2),
  (7, 2),
  (8, 2),
  (9, 2),
  (10, 2),
  (13, 2),
  (14, 2),
  (16, 2),
  (17, 2),
  (18, 2),
  (21, 2),
  (22, 2),
  (24, 2),
  (26, 2)],
 [(1, 3), (3, 3), (6, 3), (7, 3), (9, 3), (10, 3), (14, 3), (16, 3), (21, 3)],
 [(2, 4),
  (7, 4),
  (9, 4),
  (10, 4),
  (12, 4),
  (14, 4),
  (16, 4),
  (17, 4),
  (19, 4)],
 [(17, 5), (20, 5), (23, 5), (2

In [20]:
import numpy as np

def f_7(T, x_max_task_num, lambda_il):
    # 生成了每日新到达的任务?
    # T: 表示时间周期，例如天数
    # x_max_task_num: 矩阵元素的最大取值
    # lambda_il: 泊松分布的率参数矩阵 (I_citys x_max_task_num L_levels)

    # 获取 lambda_il 的维度为 I_citys 和 L_levels
    I_citys, L_levels = lambda_il.shape

    # 初始化三维数组
    arriving_tasks_i = np.zeros((T, I_citys, L_levels), dtype=int)
    
    # 生成每个时间步的 I_citys x_max_task_num L_levels 矩阵
    for t in range(T):
        for i in range(I_citys):
            for l in range(L_levels):
                # 使用泊松分布生成矩阵元素
                arriving_tasks_i[t, i, l] = min(np.random.poisson(lambda_il[i, l]), x_max_task_num)
    
    return arriving_tasks_i



In [21]:
# 示例参数
T = 7  # 时间步数量
x_max_task_num = 3  # 最大值
I_citys = 40  # 城市数量
L_levels = 5  # 等级数量
lambda_il = np.random.rand(I_citys, L_levels)  # 生成率参数矩阵

# 生成arriving_tasks_i
arriving_tasks_i = f_7(T, x_max_task_num, lambda_il)
print(arriving_tasks_i.shape, arriving_tasks_i)

(7, 40, 5) [[[0 1 1 0 2]
  [0 0 0 0 0]
  [0 0 0 1 1]
  ...
  [2 0 0 1 1]
  [1 1 1 0 1]
  [0 0 0 0 1]]

 [[0 1 1 1 2]
  [0 0 0 0 1]
  [0 0 0 0 0]
  ...
  [0 1 1 1 2]
  [0 0 1 0 0]
  [0 0 0 0 0]]

 [[2 0 1 0 0]
  [0 0 0 1 2]
  [1 1 0 0 0]
  ...
  [1 0 0 0 1]
  [1 2 0 0 0]
  [3 0 1 1 1]]

 ...

 [[1 0 1 1 0]
  [0 1 0 0 0]
  [0 0 0 2 0]
  ...
  [1 1 0 1 2]
  [0 0 0 0 0]
  [1 0 0 0 2]]

 [[3 0 3 0 0]
  [0 0 0 0 0]
  [2 0 0 1 0]
  ...
  [0 0 0 0 0]
  [2 0 1 1 0]
  [0 1 0 0 0]]

 [[3 0 0 0 2]
  [0 0 0 0 0]
  [0 1 1 0 0]
  ...
  [1 0 0 0 1]
  [1 1 1 0 0]
  [0 0 0 0 1]]]


In [22]:
func1 = generate_state
func2 = aggreg_state
func3 = func3_fix
func4 = f_4
func5 = func5
# func6 = 
func7 = f_7

In [23]:
import numpy as np
import copy

def func8(f1, f2, f6, f7, V0, max_iter, I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd, Z_cluster_num, X, mathcal_L, mathscr_L, N_1, N_2, r1, c1, c2):
    """
    带起始探索的强化学习算法,生成最优值函数和最优策略。

    参数:
    f1 (function): 随机生成状态的函数。
    f2 (function): 状态压缩函数。
    f6 (function): 生成决策空间的函数。
    f7 (np.array): 每个时间段新增任务的数量矩阵。
    V0 (list): 初始值函数,全为0。
    max_iter (int): 最大迭代次数。
    I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd, Z_cluster_num, X, mathcal_L, mathscr_L, N_1, N_2, r1, c1, c2: 
        函数1、2、5、6需要的参数,与之前一致。

    返回:
    tuple: (V, pi),其中V是最优值函数,pi是最优策略。
    """
    V = copy.deepcopy(V0)  # 初始化值函数 初始令所有的 V0 = 0；
    pi = {}  # 初始化策略

    for j in range(max_iter):
        S = f1(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd)  # 随机生成初始状态
        SS, AA, RR = [], [], []  # 记录状态、决策和收益的序列

        for t in range(1, W_workdays+1):  # 对每个时间段
            A = None
            max_Q = float('-inf')
            for Y in f6(S, mathcal_L, mathscr_L, N_1, N_2):  # 遍历决策空间
                R = func5(S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1, c1, c2)[1]  # 计算即时收益
                next_S = transition(S, Y, f7)  # 计算下一个状态
                next_S_bar = f2(next_S, Z_cluster_num, X, M_servers, I_citys, L_levels)  # 状态压缩
                Q = R + V[t][hash(next_S_bar)]  # 计算动作值函数
                if Q > max_Q:
                    max_Q = Q
                    A = Y

            SS.append(S)
            AA.append(A)
            RR.append(func5(S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1, c1, c2)[1])

            S = transition(S, A, f7)  # 状态转移

        # 更新值函数
        G = 0
        for t in range(W_workdays-1, -1, -1):
            G += RR[t]  # 计算累积收益
            S_bar = f2(SS[t], Z_cluster_num, X, M_servers, I_citys, L_levels)  # 状态压缩
            n = count(S_bar, SS[:t])  # 计算状态出现次数
            V[t][hash(S_bar)] = (G + (n-1) * V[t][hash(S_bar)]) / n  # 更新值函数

    # 计算最优策略
    for S in get_all_states(I_citys, L_levels, W_workdays, M_servers, x_max_task_num):
        pi[hash(S)] = None
        max_Q = float('-inf')
        for Y in f6(S, mathcal_L, mathscr_L, N_1, N_2):
            R = func5(S, mathcal_L, mathscr_L, N_1, N_2, H_home_of_server, r1, c1, c2)[1]
            next_S = transition(S, Y, f7)
            next_S_bar = f2(next_S, Z_cluster_num, X, M_servers, I_citys, L_levels)
            Q = R + V[W_workdays-1][hash(next_S_bar)]
            if Q > max_Q:
                max_Q = Q
                pi[hash(S)] = Y

    return V, pi

def transition(S, Y, Xi):
    """
    状态转移函数。

    参数:
    S (tuple): 当前状态。
    Y (list): 当前决策。
    Xi (np.array): 每个时间段新增任务的数量矩阵。

    返回:
    tuple: 下一个状态。
    """
    n_il, servers_info = S
    n_il_new = n_il + Xi - count_tasks(Y)
    servers_info_new = []
    for m in range(len(servers_info)):
        i, w = servers_info[m]
        if w > 0:
            servers_info_new.append((Y[m][0], w-1))
        else:
            servers_info_new.append((H_home_of_server[m], W_workdays))
    return (n_il_new, servers_info_new)

def count_tasks(Y):
    """
    统计决策Y中完成的任务数量。

    参数:
    Y (list): 决策。

    返回:
    np.array: 每个城市每个等级完成的任务数量。
    """
    n_il = np.zeros((I_citys, L_levels), dtype=int)
    for m in range(M_servers):
        i, l = Y[m]
        n_il[i-1][l-1] += 1
    return n_il

def count(S_bar, SS):
    """
    统计状态S_bar在状态序列SS中出现的次数。

    参数:
    S_bar (tuple): 压缩后的状态。
    SS (list): 状态序列。

    返回:
    int: 出现次数。
    """
    return np.sum([1 for S in SS if f2(S, Z_cluster_num, X, M_servers, I_citys, L_levels) == S_bar])

def get_all_states(I_citys, L_levels, W_workdays, M_servers, x_max_task_num):
    """
    生成所有可能的状态。

    参数:
    I_citys (int): 城市数。
    L_levels (int): 等级数。
    W_workdays (int): 最大工作天数。
    M_servers (int): 服务员数。
    x_max_task_num (int): 每个城市每个等级的最大任务数。

    返回:
    list: 所有可能的状态列表。
    """
    states = []
    for n_il in itertools.product(range(x_max_task_num+1), repeat=I_citys*L_levels):
        for servers_info in itertools.product(itertools.product(range(1, I_citys+1), range(W_workdays+1)), repeat=M_servers):
            states.append((np.array(n_il).reshape((I_citys, L_levels)), list(servers_info)))
    return states

def hash(S):
    """
    状态的哈希函数,将状态映射为整数。

    参数:
    S (tuple): 状态。

    返回:
    int: 状态的哈希值。
    """
    return hash(str(S))

: 

In [54]:
# 示例参数
T = 7  # 时间步数量
x_max_task_num = 3  # 最大值
I_citys = 26  # 城市数量
L_levels = 5  # 等级数量
lambda_il = np.random.rand(I_citys, L_levels)  # 生成率参数矩阵

# 生成arriving_tasks_i
arriving_tasks_i = func7(T, x_max_task_num, lambda_il)
print(arriving_tasks_i.shape, arriving_tasks_i)

(7, 26, 5) [[[0 1 0 0 0]
  [0 0 0 0 0]
  [1 1 0 0 1]
  [0 1 1 0 1]
  [0 2 0 0 3]
  [0 0 1 0 1]
  [0 0 0 0 2]
  [0 1 1 1 0]
  [1 0 2 0 0]
  [0 0 0 0 2]
  [1 0 0 0 0]
  [0 1 0 0 0]
  [0 0 1 0 0]
  [1 0 1 2 0]
  [0 3 0 0 0]
  [1 0 1 0 0]
  [0 0 0 1 2]
  [1 0 1 2 2]
  [0 0 0 1 1]
  [0 0 1 0 0]
  [0 0 0 1 0]
  [0 1 0 1 0]
  [0 0 0 0 0]
  [3 0 0 0 0]
  [0 1 2 1 0]
  [0 0 0 1 2]]

 [[0 1 1 0 2]
  [0 0 0 0 0]
  [0 0 0 0 1]
  [0 0 1 0 0]
  [0 3 0 0 0]
  [0 0 1 0 0]
  [2 1 0 1 1]
  [1 1 0 0 2]
  [0 0 0 0 0]
  [1 1 0 0 1]
  [0 0 0 0 0]
  [2 0 0 0 0]
  [3 0 0 2 1]
  [0 0 2 1 0]
  [1 0 0 3 0]
  [1 1 0 1 0]
  [0 0 0 0 0]
  [1 0 0 1 1]
  [0 0 0 0 3]
  [0 0 1 0 0]
  [1 0 1 1 0]
  [0 2 0 0 2]
  [0 0 0 0 0]
  [0 0 1 0 0]
  [1 0 0 0 0]
  [0 0 0 1 1]]

 [[0 0 1 1 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [2 0 0 0 0]
  [0 0 0 0 1]
  [0 1 1 0 0]
  [1 1 0 1 0]
  [0 0 1 0 0]
  [0 2 1 0 0]
  [0 0 2 0 0]
  [2 2 0 0 0]
  [0 0 1 1 1]
  [0 0 0 0 0]
  [0 0 0 1 1]
  [0 2 1 0 3]
  [0 1 0 0 1]
  [2 1 0 0 0]
  [0 

In [ ]:
#函数8
import numpy as np



for j in range(J):
    s0 = func1(I_citys, L_levels, W_workdays, M_servers, x_max_task_num, H_home_of_server, lambd)
    xi = func7(T, x_max_task_num, lambda_il)
    R=[0]*T
    for t in range(T):
        action = func3(s0, L_mathscr, H_home_of_server, r1, c1, c2)
        R[t] = revenue(s0,action)
        s1 = state_trans(s0,action,xi[t])
        
        



def revenue(st,at):
    


def state_trans(S0,act,xi):   #状态转移
    dic1 = {}
    for i, row in enumerate(S0):
        for j, value in enumerate(row):
            dic1[(i+1, j+1)] = value
    dic2 = {(x[1], x[2]): 1 for x in act if x[2] != 0}
    S_A_cell = {}
    for key in dic1:
        if key in dic2:
            S_A_cell[key] = dic1[key] - dic2[key]
        else:
            S_A_cell[key] = dic1[key]
    S_A = [[0] *len(S0[0]) for _ in range(len(S0))]
    for key, value in S_A_cell.items():
        S_A[key[0]-1][key[1]-1] = value
        
    result = np.add(S_A,xi)
    return result

In [55]:
import numpy as np

def state_trans(S0,act,xi):
    dic1 = {}
    for i, row in enumerate(S0):
        for j, value in enumerate(row):
            dic1[(i+1, j+1)] = value
    dic2 = {(x[1], x[2]): 1 for x in act if x[2] != 0}
    S_A_cell = {}
    for key in dic1:
        if key in dic2:
            S_A_cell[key] = dic1[key] - dic2[key]
        else:
            S_A_cell[key] = dic1[key]
    S_A = [[0] *len(S0[0]) for _ in range(len(S0))]
    for key, value in S_A_cell.items():
        S_A[key[0]-1][key[1]-1] = value
        
    result = np.add(S_A,xi)
    return result

In [56]:
S=state[0]
act = A
xi = arriving_tasks_i[0]
tra = state_trans(S,act,xi)
print(tra)

[[ 0  1  0  1  0]
 [ 0  2  0  3  0]
 [ 1  2  1  3  3]
 [ 0  0  3  0  2]
 [ 0  3  0  2  3]
 [ 0 -1  1  0  1]
 [-1  1  0  1  2]
 [ 1  1  2  2  0]
 [ 1  0  2  0  0]
 [-1  0  1  1  3]
 [ 2  0  0  2  1]
 [ 0  1  0  0  1]
 [ 0 -1  1  0  0]
 [ 0  1  1  3  0]
 [ 0  4  0  0  0]
 [ 1  0  0  0  0]
 [ 1  0  1  1  4]
 [ 3  0  0  3  2]
 [ 0  0  1  1  1]
 [-1  0  1  0  0]
 [ 1  0  0  3  0]
 [ 0  0 -1  2  0]
 [ 0  1  2  1  0]
 [ 3  0  0  0 -1]
 [ 0  1  2  1  1]
 [ 0  0  0  2  3]]
